In [6]:
import numpy as np
import re
import matplotlib.pyplot as plt
import polars as pl
import seaborn as sns

In [43]:
# Setup and global variables
# --------------------------
# Set the path to the data directory
fig_dir = "img/"
# Plotting setup and style
custom_cmap = plt.get_cmap("viridis")
sns.set_style("darkgrid")
# Relative data paths
output_energy = "output-energy" # Energy measurement output file
output_timing = "output-timing" # Timing measurement output file
s_hardware = "-hardware"
s_software = "-software"
s_full = "-full"

# Constants
class Energy:
    CURRENT = {
        "CPU": 1.8, # mA as per https://www.willow.co.uk/TelosB_Datasheet.pdf
        "LPM": 0.0051 + 0.021, # 26.1 uA as per https://www.willow.co.uk/TelosB_Datasheet.pdf
        "DEEP_LPM": 0.0061, # 6.1 uA as per https://www.willow.co.uk/TelosB_Datasheet.pdf
        "RADIO_RX": 18.8, # https://www.ti.com/lit/ds/symlink/cc2420.pdf?ts=1670166938154&ref_url=https%253A%252F%252Fwww.ti.com%252Fproduct%252FCC2420
        "RADIO_TX": 17.4 # https://www.ti.com/lit/ds/symlink/cc2420.pdf?ts=1670166938154&ref_url=https%253A%252F%252Fwww.ti.com%252Fproduct%252FCC2420
    }
    VOLTAGE = 3
    TICKS_PER_SECOND = 32768

In [45]:
"""
Takes a path to a file containing the timing measurements
Returns a DataFrame with the timing measurements
"""
def extract_timings(path: str) -> pl.DataFrame:
    """Extract the timings from the output timing file"""
    # Read the file
    with open(path, "r") as f:
        file = f.read()
    # Extract Sensor timings
    sensor_timings = re.findall(r"Sensor time: (\d+)", file)
    sensor_timings = [int(t) for t in sensor_timings]
    # Extract AES encryption timings
    aes_timings = re.findall(r"AES time: (\d+)", file)
    aes_timings = [int(t) for t in aes_timings]
    # Extract TX timings
    tx_timings = re.findall(r"TX time: (\d+)", file)
    tx_timings = [int(t) for t in tx_timings]

    # Create a DataFrame
    df = pl.DataFrame(
        {
            "Sensor": sensor_timings,
            "AES Encryption": aes_timings,
            "TX": tx_timings,
        }
    )
    return df

"""
Takes a summary string and returns a list of values
Returns a list of values
"""
def extract_values(summary: str, samples=1) -> list:
    # extract the values
    match = re.findall(r":\s+(\d+)", summary)
    # return the values
    return [int(x) / samples if i > 0 else int(x) for i, x in enumerate(match)]

"""
Takes a path to a file containing the energy measurements
Returns a DataFrame with the energy measurements
"""
def extract_summaries(energest_output: str) -> list:
    # open the file to apply regex on
    with open(energest_output, "r") as f:
        # read the file
        file = f.read()
        # extract the summaries
        # summaries = summary_regex.findall(file)
        summaries = re.findall(r"Sensor energy:.*?Radio total\s+:\s+\d+/\s+\d+ \(\d+ permil\)", file, re.DOTALL)
        # return the summaries
        return summaries

In [47]:
df = extract_timings(output_timing + s_software + ".txt")
len(df)

ShapeError: Could not create a new DataFrame from Series. The Series have different lengths. Got [shape: (178,)
Series: 'Sensor' [i64]
[
	54
	55
	55
	55
	55
	55
	55
	55
	55
	55
	54
	54
	...
	55
	55
	55
	55
	55
	56
	54
	54
	54
	54
	54
	54
	54
], shape: (177,)
Series: 'AES Encryption' [i64]
[
	112
	115
	115
	115
	115
	114
	114
	115
	114
	114
	112
	114
	...
	114
	114
	114
	114
	114
	114
	112
	114
	114
	114
	114
	114
	114
], shape: (177,)
Series: 'TX' [i64]
[
	120
	122
	122
	122
	121
	122
	121
	122
	121
	122
	122
	122
	...
	122
	122
	122
	122
	122
	122
	121
	122
	122
	121
	121
	122
	122
]]

In [50]:
df = extract_summaries(output_energy + s_hardware + ".txt")
len(df)

1060